In [109]:
import yfinance as yf
import numpy as np
import random

np.random.seed(42)
random.seed(42)

In [110]:
# 計算年化報酬
def calculate_annual_return(data):
    # 計算股票的年化報酬
    total_days = (data.index[-1] - data.index[0]).days  # 總天數
    years = total_days / 365.0  # 轉換成年
    start_value = data.iloc[0]  # 開盤價
    end_value = data.iloc[-1]  # 收盤價
    return (end_value / start_value) ** (1 / years) - 1  # 計算年化報酬

In [111]:
# 計算年化波動率
def calculate_annual_risk(data):
    daily_returns = data.pct_change().dropna()  # 計算每日變動
    return daily_returns.std() * np.sqrt(len(daily_returns))  # 計算年化波動

In [135]:
# 抓取股市資料
def get_stock_data(stocks, start_date, end_date):
    stock_returns = []  # 存儲每支股票的報酬
    stock_risks = []  # 存儲每支股票的風險
    for stock_id in stocks:
        data = yf.download(stock_id, start=start_date, end=end_date)['Adj Close']  # 下載股市資料
        annual_return = calculate_annual_return(data)  # 計算報酬
        annual_risk = calculate_annual_risk(data)  # 計算風險
        stock_returns.append(annual_return)  # 添加到報酬清單
        stock_risks.append(annual_risk)  # 添加到風險清單
    return np.array(stock_returns), np.array(stock_risks)

In [136]:
# 定義股市資料範圍與股票符號
start_date = '2020-01-01'
end_date = '2024-01-01'

#台積電、聯發科、中華電信、長榮航、國泰金控、台泥、台灣塑膠、統一超商、中鋼、群創
stocks_list = ['2330.TW', '2454.TW', '2412.TW', '2618.TW', '2882.TW',
        '1101.TW', '1301.TW', '2912.TW', '2002.TW', '3481.TW']

# 取得所有股票的報酬和風險資料
stock_returns, stock_risks = get_stock_data(stocks_list, start_date, end_date)


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [137]:
# 計算最大回撤
def calculate_max_drawdown(data):
    cumulative_returns = (1 + data).cumprod()  # 計算累積回報
    peak = cumulative_returns.cummax()  # 計算歷史上的最高值
    drawdown = (cumulative_returns - peak) / peak  # 計算回撤
    max_drawdown = drawdown.min()  # 最大回撤是最小的回撤值
    return max_drawdown


In [138]:
# 基因演算法的基本設定
population_size = 20  # 族群大小
num_generations = 200  # 世代數
mutation_rate = 0.3  # 突變率
num_stocks = len(stocks_list)  # 股票數量

In [145]:
# 適應度函數，用來計算每個投資組合的表現
def fitness(chromosome):
    portfolio_return = np.sum(chromosome * stock_returns)  # 計算投資組合的回報
    portfolio_risk = np.sqrt(np.sum((chromosome * stock_risks) ** 2))  # 計算投資組合的風險
    stock_count = np.sum(chromosome)  # 計算選中的股票數量

    if portfolio_risk > 0:  # 避免風險為0的情況
        sharpe_ratio = portfolio_return / portfolio_risk  # 計算夏普比率
    else:
        sharpe_ratio = 0

    # 目標是最大化夏普比率並減少選股數量
    return 0.8 * sharpe_ratio - 0.006 * stock_count

In [140]:
# 初始化族群
def initialize_population():
    return [np.random.randint(0, 2, num_stocks) for _ in range(population_size)]  # 隨機生成族群


In [141]:
# 選擇操作，根據適應度選擇較好的個體
def select(population, fitness_scores):
    min_fitness = min(fitness_scores)  # 找到最小的適應度
    shifted_fitness = [f - min_fitness + 1e-6 for f in fitness_scores]  # 平移適應度，使其不為負
    total_fitness = sum(shifted_fitness)  # 計算總適應度
    probs = [f / total_fitness for f in shifted_fitness]  # 計算每個個體被選中的機率
    return population[np.random.choice(range(population_size), p=probs)]  # 按照機率選擇父母


In [142]:
# 交配操作，將兩個父母的基因組合成新個體
def crossover(parent1, parent2):
    point = np.random.randint(1, num_stocks - 1)  # 隨機選擇交配點
    child1 = np.concatenate((parent1[:point], parent2[point:]))  # 第一個小孩
    child2 = np.concatenate((parent2[:point], parent1[point:]))  # 第二個小孩
    return child1, child2

In [127]:
# 突變操作，隨機改變個體的基因
def mutate(chromosome):
    for i in range(num_stocks):
        if random.random() < mutation_rate:  # 根據突變率隨機改變基因
            chromosome[i] = 1 - chromosome[i]  # 基因翻轉
    return chromosome

In [143]:
# 基因演算法的主程式
def genetic_algorithm():
    population = initialize_population()  # 初始化族群
    for generation in range(num_generations):  # 進行多代演化
        fitness_scores = [fitness(chromosome) for chromosome in population]  # 計算適應度
        new_population = []  # 用來存儲新一代的族群
        for _ in range(population_size // 2):  # 每次交配兩個父母
            parent1 = select(population, fitness_scores)  # 選擇第一個父母
            parent2 = select(population, fitness_scores)  # 選擇第二個父母
            child1, child2 = crossover(parent1, parent2)  # 交配產生小孩
            child1 = mutate(child1)  # 突變小孩1
            child2 = mutate(child2)  # 突變小孩2
            new_population.append(child1)  # 加入新族群
            new_population.append(child2)
        population = new_population  # 更新族群
        best_fitness = max(fitness_scores)  # 取出最好的適應度
        print(f"Generation {generation + 1}: Best Fitness = {best_fitness}")  # 印出進度
    best_index = np.argmax([fitness(chromosome) for chromosome in population])  # 找到最好的組合
    return population[best_index]

In [146]:
# 執行基因演算法
best_portfolio = genetic_algorithm()
print("\n最佳選股組合:", best_portfolio)
print("總報酬:", np.sum(best_portfolio * stock_returns))  # 計算總回報
print("總風險:", np.sqrt(np.sum((best_portfolio * stock_risks) ** 2)))  # 計算總風險


Generation 1: Best Fitness = 1.3548441598827692
Generation 2: Best Fitness = 1.270240553329512
Generation 3: Best Fitness = 1.433940927601577
Generation 4: Best Fitness = 1.354844159882769
Generation 5: Best Fitness = 1.354844159882769
Generation 6: Best Fitness = 1.3548441598827692
Generation 7: Best Fitness = 1.3548441598827694
Generation 8: Best Fitness = 1.433940927601577
Generation 9: Best Fitness = 1.433940927601577
Generation 10: Best Fitness = 1.3548441598827694
Generation 11: Best Fitness = 1.3548441598827694
Generation 12: Best Fitness = 1.3548441598827694
Generation 13: Best Fitness = 1.4339409276015769
Generation 14: Best Fitness = 1.3548441598827694
Generation 15: Best Fitness = 1.270240553329512
Generation 16: Best Fitness = 1.4339409276015769
Generation 17: Best Fitness = 1.4339409276015769
Generation 18: Best Fitness = 1.354844159882769
Generation 19: Best Fitness = 1.3548441598827694
Generation 20: Best Fitness = 1.3548441598827694
Generation 21: Best Fitness = 1.27024

In [147]:
# 顯示結果
def summarize_results(best_portfolio):
    print("最佳選股組合:", best_portfolio)
    print("總報酬:", np.sum(best_portfolio * stock_returns))
    print("總風險:", np.sqrt(np.sum((best_portfolio * stock_risks) ** 2)))
    print("\n股票的選擇狀態:")
    for i, stock in enumerate(stocks_list):
        print(f"{stock}: {'選擇' if best_portfolio[i] == 1 else '不選擇'}")

summarize_results(best_portfolio)  # 顯示最終的投資組合選擇

最佳選股組合: [1 1 1 0 1 1 0 1 1 1]
總報酬: 9.08894504373214
總風險: 5.183153084939482

股票的選擇狀態:
2330.TW: 選擇
2454.TW: 選擇
2412.TW: 選擇
2618.TW: 不選擇
2882.TW: 選擇
1101.TW: 選擇
1301.TW: 不選擇
2912.TW: 選擇
2002.TW: 選擇
3481.TW: 選擇
